In [ ]:
# mount google colab

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# install packages

!pip install transformers datasets evaluate
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.1 MB/s eta 0:00:00


In [ ]:
# login to huggingface
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# import dataset wich should be used for finetuning
from datasets import load_dataset

left_heading_dataset = load_dataset('csv', data_files='add_path_to_dataset/allsides_data_left_fulltext.csv')
left_heading_dataset = left_heading_dataset['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# split data into trainings and test split
left_heading_dataset = left_heading_dataset.train_test_split(test_size=0.2)

# Preprocess

In [ ]:
# import tokenizer for the pretrained model wich should be used
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

In [ ]:
# function to apply tokenizing
def preprocess_function(examples):
    return tokenizer(examples["full_text"]) #define column wich should be used for finetuning

In [ ]:
# tokenize dataset
tokenized_left_heading_dataset = left_heading_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=left_heading_dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/8540 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1291 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1384 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1353 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1328 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/2136 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1894 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1878 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1853 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1395 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
# chunk size for concatenation
block_size = 128

# function to concatenate data
def group_texts(examples):
    # concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # split by chunks of block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_dataset = tokenized_left_heading_dataset.map(group_texts, batched=True, num_proc=4)
#lm_dataset['train'][0]

Map (num_proc=4):   0%|          | 0/8540 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2136 [00:00<?, ? examples/s]

In [ ]:
# import the Trainer and Datacollector for finetuning purposes
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

# add pad token to tokenizer
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #load datacollector

# Train

In [ ]:
# import the pretrained model wich should be finetuned
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

In [ ]:
#finetune the pretrained model on the dataset
training_args = TrainingArguments(
    output_dir="distillgpt2_left_fulltext_torch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    num_train_epochs=6
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

Cloning https://huggingface.co/tobijen/distillgpt2_left_fulltext_torch into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,3.236300,3.151246
2,3.174300,3.128597
3,3.124400,3.114672
4,3.101800,3.110237
5,3.085200,3.105682
6,3.064600,3.104150


TrainOutput(global_step=59280, training_loss=3.140524461877491, metrics={'train_runtime': 6711.6008, 'train_samples_per_second': 70.653, 'train_steps_per_second': 8.832, 'total_flos': 1.5488299567153152e+16, 'train_loss': 3.140524461877491, 'epoch': 6.0})

In [ ]:
# get the perplexity for the model
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 22.29


In [ ]:
# push finetuned model to huggingface repo
trainer.push_to_hub()

In [ ]:
# save model locally
trainer.save_model("path_to_where_the_model_should_be_saved_locally/distillgpt2_left_fulltext_torch")

Upload file pytorch_model.bin:   0%|          | 1.00/312M [00:00<?, ?B/s]

Upload file runs/Jul26_20-07-11_8f4e0f613aae/events.out.tfevents.1690402042.8f4e0f613aae.5756.0:   0%|        …

Upload file runs/Jul26_20-07-11_8f4e0f613aae/events.out.tfevents.1690408806.8f4e0f613aae.5756.1:   0%|        …

To https://huggingface.co/tobijen/distillgpt2_left_fulltext_torch
   64546cf..2ffd7c5  main -> main

   64546cf..2ffd7c5  main -> main

To https://huggingface.co/tobijen/distillgpt2_left_fulltext_torch
   2ffd7c5..faea163  main -> main

   2ffd7c5..faea163  main -> main



# Inference

In [ ]:
# Inputs to generate text for
prompts = {
    "social_1": "Gay marriage is",
    "social_2": "Abortion is",
    "education_1": "Public education is",
    "education_2": "Charter schools are",
    "economic_1":"The increase of taxes is",
    "economic_2":"Government regulations on businesses are",
}

In [ ]:
from transformers import pipeline
# generate text with the pipeline module from the transformers library
generated_text_dict = {}
for key, prompt in prompts.items():
  print(key, " => ", prompt)
  generator = pipeline("text-generation", model="tobijen/distillgpt2_left_fulltext_torch", tokenizer=tokenizer)
  generated_text = generator(prompt, return_full_text=True, max_new_tokens=100)
  generated_text_dict[key] = generated_text[0]["generated_text"]
  print(generated_text)

social_1  =>  Gay marriage is


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Gay marriage is legal nationwide, a landmark ruling in the landmark landmark federal case, which struck down the Supreme Court’s landmark 1973 ruling declaring an individual who has a right to marry is not entitled to same-sex marriage or the right to a same-sex civil partnership.\n\nIn the case, Supreme Court Justice Sonia Sotomayor cited the court’s previous rulings to highlight that it had the power over same-sex marriage but not the same rights to same-sex marriagerunner marriages.'}]
social_2  =>  Abortion is


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Abortion is illegal in the US. Although the federal government recognizes this case, the Supreme Court found a similar case in 2013 in federal district court Judge Wroten.\n\nBurgess Everett, a Republican, argued against the case, arguing the government\'s attempts to exclude the patient could be an infringement of a patient\'s right to life.\n\n"Although that case is an attempt to put a patient through a difficult situation and put her on high alert, we found there are limits on the ability of'}]
education_1  =>  Public education is


...

[{'generated_text': "Charter schools are in a strong position because of the rising cost of vaccinations on behalf of students, and the drop in Covid-19 cases among children age 5 to 14, as well as due to the increasing number of new cases around Washington, D.C.\n\nPence and other administration officials stressed that this is not the first time boosters will be needed.\n\n“All kids have got to go through the same process all their age groups,” said Dr. Philip S. O'"}]
economic_1  =>  The increase of taxes is


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The increase of taxes is part of a wider trend toward lower-income earners. For example, a 2009 law that capped deductions for the top 1 percent of income, for instance, raised the top $5,000 to $3,400 a year. Tax experts noted that the hike is largely motivated by higher minimum tax rates and lower tax rates to encourage higher earners to invest in higher-paying jobs.\n\nBut economists say the tax increases are unlikely to translate to higher-income Americans’ pocketbooks. Vieira'}]
economic_2  =>  Government regulations on businesses are


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Government regulations on businesses are not based on the content or practices of government," the statement said.\n\nLawmakers have been calling for a public review of the guidance, which has been criticized by the president for its lack of rigor.\n\n"Let\'s consider a range of recommendations to help people to prepare for and mitigate harm to themselves by providing tools that help people to safely and professionally access resources for work," the Democratic-led Senate said in a statement. "While many Americans do not fully understand how the guidance'}]


In [ ]:
generated_text_dict

{'social_1': 'Gay marriage is legal nationwide, a landmark ruling in the landmark landmark federal case, which struck down the Supreme Court’s landmark 1973 ruling declaring an individual who has a right to marry is not entitled to same-sex marriage or the right to a same-sex civil partnership.\n\nIn the case, Supreme Court Justice Sonia Sotomayor cited the court’s previous rulings to highlight that it had the power over same-sex marriage but not the same rights to same-sex marriagerunner marriages.',
 'social_2': 'Abortion is illegal in the US. Although the federal government recognizes this case, the Supreme Court found a similar case in 2013 in federal district court Judge Wroten.\n\nBurgess Everett, a Republican, argued against the case, arguing the government\'s attempts to exclude the patient could be an infringement of a patient\'s right to life.\n\n"Although that case is an attempt to put a patient through a difficult situation and put her on high alert, we found there are limi

In [ ]:
# add the generated text to a json file, wich is used for the evaluation
import json
import os

def write_dict_to_json_file(file_path, data):
    # Check if the JSON file exists
    if os.path.exists(file_path):
        # If the file exists, load the existing data
        with open(file_path, 'r', encoding='utf-8') as file:
            existing_data = json.load(file)
    else:
        # If the file does not exist, create an empty dictionary
        existing_data = {}

    # Update the existing dictionary with the new data
    existing_data.update(data)

    # Write the updated dictionary back to the JSON file
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False)

data_to_append = {
    "distilgpt2_left_fulltext": generated_text_dict
}

# write data to json file
write_dict_to_json_file('path_to_json_file/generated_texts.json', data_to_append)